In [1]:
from graphframes import GraphFrame
from pixiedust.display import *

Pixiedust database opened successfully


In [2]:
nodes = spark.read.csv('/home/maria/Documents/grafos/transport-nodes.csv', header=True)
rels = spark.read.csv('/home/maria/Documents/grafos/transport-relationships.csv', header=True)


In [3]:
nodes=nodes.withColumn("population", nodes["population"].cast("double"))

In [4]:
rels=rels.withColumn("cost", rels["cost"].cast("Int"))

In [5]:
rels.show(2,False)

+---------+--------+------------+----+
|src      |dst     |relationship|cost|
+---------+--------+------------+----+
|Amsterdam|Utrecht |EROAD       |46  |
|Amsterdam|Den Haag|EROAD       |59  |
+---------+--------+------------+----+
only showing top 2 rows



In [6]:
g = GraphFrame(nodes, rels)

In [7]:
g.vertices.show()
g.edges.show()
## Check the number of edges of each vertex
g.degrees.show()


+----------------+---------+---------+----------+
|              id| latitude|longitude|population|
+----------------+---------+---------+----------+
|       Amsterdam|52.379189| 4.899431|  821752.0|
|         Utrecht|52.092876| 5.104480|  334176.0|
|        Den Haag|52.078663| 4.288788|  514861.0|
|       Immingham| 53.61239| -0.22219|    9642.0|
|       Doncaster| 53.52285| -1.13116|  302400.0|
|Hoek van Holland|  51.9775|  4.13333|    9382.0|
|      Felixstowe| 51.96375|   1.3511|   23689.0|
|         Ipswich| 52.05917|  1.15545|  133384.0|
|      Colchester| 51.88921|  0.90421|  104390.0|
|          London|51.509865|-0.118092| 8787892.0|
|       Rotterdam|  51.9225|  4.47917|  623652.0|
|           Gouda| 52.01667|  4.70833|   70939.0|
+----------------+---------+---------+----------+

+----------------+----------------+------------+----+
|             src|             dst|relationship|cost|
+----------------+----------------+------------+----+
|       Amsterdam|         Utrecht|  

In [ ]:
display(g)

In [10]:
# Obtain list of direct flights between SEA and SFO
filteredPaths = g.bfs(
  fromExpr = "id = 'London'",
  toExpr = "id = 'Colchester'")

# display list of direct flights
#display(filteredPaths)
filteredPaths.show(1,False)

+-----------------------------------------+--------------------------------+-----------------------------------------+
|from                                     |e0                              |to                                       |
+-----------------------------------------+--------------------------------+-----------------------------------------+
|[London, 51.509865, -0.118092, 8787892.0]|[London, Colchester, EROAD, 106]|[Colchester, 51.88921, 0.90421, 104390.0]|
+-----------------------------------------+--------------------------------+-----------------------------------------+



In [12]:
filteredPaths = g.bfs(
  fromExpr = "id = 'Amsterdam'",
  toExpr = "id = 'Utrecht'")
filteredPaths.show(5,False)

+------------------------------------------+-------------------------------+----------------------------------------+
|from                                      |e0                             |to                                      |
+------------------------------------------+-------------------------------+----------------------------------------+
|[Amsterdam, 52.379189, 4.899431, 821752.0]|[Amsterdam, Utrecht, EROAD, 46]|[Utrecht, 52.092876, 5.104480, 334176.0]|
+------------------------------------------+-------------------------------+----------------------------------------+



In [13]:
filteredPaths = g.bfs(
  fromExpr = "id = 'Amsterdam'",
  toExpr = "id = 'Immingham'")
filteredPaths.show(5,False)

+------------------------------------------+----------------------------------+---------------------------------------+
|from                                      |e0                                |to                                     |
+------------------------------------------+----------------------------------+---------------------------------------+
|[Amsterdam, 52.379189, 4.899431, 821752.0]|[Amsterdam, Immingham, EROAD, 369]|[Immingham, 53.61239, -0.22219, 9642.0]|
+------------------------------------------+----------------------------------+---------------------------------------+



In [14]:
filteredPaths = g.bfs(
  fromExpr = "id = 'Amsterdam'",
  toExpr = "id = 'Rotterdam'")
filteredPaths.show(5,False)

+------------------------------------------+--------------------------------+-----------------------------------------+--------------------------------+---------------------------------------+
|from                                      |e0                              |v1                                       |e1                              |to                                     |
+------------------------------------------+--------------------------------+-----------------------------------------+--------------------------------+---------------------------------------+
|[Amsterdam, 52.379189, 4.899431, 821752.0]|[Amsterdam, Den Haag, EROAD, 59]|[Den Haag, 52.078663, 4.288788, 514861.0]|[Den Haag, Rotterdam, EROAD, 26]|[Rotterdam, 51.9225, 4.47917, 623652.0]|
+------------------------------------------+--------------------------------+-----------------------------------------+--------------------------------+---------------------------------------+



In [15]:
filteredPaths = g.bfs(
  fromExpr = "id = 'Gouda'",
  toExpr = "id = 'Utrecht'")
filteredPaths.show(5,False)

+-----------------------------------+---------------------------+----------------------------------------+
|from                               |e0                         |to                                      |
+-----------------------------------+---------------------------+----------------------------------------+
|[Gouda, 52.01667, 4.70833, 70939.0]|[Gouda, Utrecht, EROAD, 35]|[Utrecht, 52.092876, 5.104480, 334176.0]|
+-----------------------------------+---------------------------+----------------------------------------+



In [20]:
filteredPaths=g.find("(London)-[LondonColchester]->(Colchester); (Colchester)-[ColchesterIpswich]->(Ipswich)")

In [21]:
filteredPaths.show(15,False)

+--------------------------------------------+------------------------------------------+--------------------------------------------+------------------------------------------+--------------------------------------------+
|London                                      |LondonColchester                          |Colchester                                  |ColchesterIpswich                         |Ipswich                                     |
+--------------------------------------------+------------------------------------------+--------------------------------------------+------------------------------------------+--------------------------------------------+
|[Amsterdam, 52.379189, 4.899431, 821752.0]  |[Amsterdam, Den Haag, EROAD, 59]          |[Den Haag, 52.078663, 4.288788, 514861.0]   |[Den Haag, Gouda, EROAD, 32]              |[Gouda, 52.01667, 4.70833, 70939.0]         |
|[Amsterdam, 52.379189, 4.899431, 821752.0]  |[Amsterdam, Den Haag, EROAD, 59]          |[Den Haag, 52.07866

In [17]:
filteredPaths=g.find("(Gouda)-[GoudaRotterdam]->(Rotterdam)")

In [19]:
filteredPaths.show(20,False)

+--------------------------------------------+------------------------------------------+--------------------------------------------+
|Gouda                                       |GoudaRotterdam                            |Rotterdam                                   |
+--------------------------------------------+------------------------------------------+--------------------------------------------+
|[Amsterdam, 52.379189, 4.899431, 821752.0]  |[Amsterdam, Utrecht, EROAD, 46]           |[Utrecht, 52.092876, 5.104480, 334176.0]    |
|[Amsterdam, 52.379189, 4.899431, 821752.0]  |[Amsterdam, Den Haag, EROAD, 59]          |[Den Haag, 52.078663, 4.288788, 514861.0]   |
|[Den Haag, 52.078663, 4.288788, 514861.0]   |[Den Haag, Rotterdam, EROAD, 26]          |[Rotterdam, 51.9225, 4.47917, 623652.0]     |
|[Amsterdam, 52.379189, 4.899431, 821752.0]  |[Amsterdam, Immingham, EROAD, 369]        |[Immingham, 53.61239, -0.22219, 9642.0]     |
|[Immingham, 53.61239, -0.22219, 9642.0]     |[Immingha

In [ ]:

+----------------+---------+---------+----------+
|              id| latitude|longitude|population|
+----------------+---------+---------+----------+
|       Amsterdam|52.379189| 4.899431|  821752.0|
|         Utrecht|52.092876| 5.104480|  334176.0|
|        Den Haag|52.078663| 4.288788|  514861.0|

In [31]:
g2 = g.filterVertices("population >= 514861").dropIsolatedVertices()
display(g2.vertices)

id,latitude,longitude,population
Amsterdam,52.379189,4.899431,821752.0
Den Haag,52.078663,4.288788,514861.0
Rotterdam,51.9225,4.47917,623652.0


In [32]:
g3 = g.filterVertices("population < 514861").dropIsolatedVertices()
display(g3.vertices)

id,latitude,longitude,population
Utrecht,52.092876,5.104480,334176.0
Immingham,53.61239,-0.22219,9642.0
Doncaster,53.52285,-1.13116,302400.0
Hoek van Holland,51.9775,4.13333,9382.0
Felixstowe,51.96375,1.3511,23689.0
Ipswich,52.05917,1.15545,133384.0
Colchester,51.88921,0.90421,104390.0
Gouda,52.01667,4.70833,70939.0
